In [1]:
import spatialdata as sd
import anndata as ad
import pandas as pd
import dask.dataframe as dd
import numpy as np

from thalamus_merfish_analysis import abc_load as abc

import ccf_registration as ccf

In [2]:
import matplotlib.pyplot as plt
import spatialdata_plot
get_ipython().run_line_magic('matplotlib', 'inline') 

## load and apply transforms

Simplest method for script

In [3]:
from ccf_transforms import *

coords = ['x_section', 'y_section', 'z_section']
slice_label = 'slice_int'

df_full = abc.get_combined_metadata()

In [4]:
df = abc.label_thalamus_spatial_subset(df_full, flip_y=False, distance_px=25, 
                                  cleanup_mask=True, drop_end_sections=True,
                                  filter_cells=True)

In [5]:
# ref_subset = abc.get_thalamus_reference_ids()
# df = ccf.subset_to_ref_bounds(df_full, coords, ref_subset)
# minmax = df[coords].agg(['min','max'])
# minmax.to_csv("resources/brain3_thalamus_coordinate_bounds.csv")
# minmax

In [6]:
minmax = pd.read_csv("/code/resources/brain3_thalamus_coordinate_bounds.csv", index_col=0)
min_xy, max_xy = minmax.loc['min'].values, minmax.loc['max'].values

In [7]:
# df = df_full
# df = df.loc[lambda df: 
#     (df[coords] <= max_xy).all(axis=1) &
#     (df[coords] >= min_xy).all(axis=1)]

In [8]:

df[slice_label] = df['z_section'].apply(lambda x: int(x*10))
transforms_by_section = ccf.read_quicknii_file("resources/adjusted_10-10_final.json", scale=25)
norm_transform = get_normalizing_transform(
                                           min_xy=min_xy, 
                                           max_xy=max_xy, 
                                           flip_y=True)

# load to spatialdata
cells_by_section = parse_cells_by_section(df, transforms_by_section, norm_transform, coords, slice_label=slice_label)
sdata = sd.SpatialData.from_elements_dict(cells_by_section)

In [9]:
# transform
transformed_points = pd.concat((
    df.compute() for df in 
    sdata.transform_to_coordinate_system('ccf').points.values()
    ))

# update dataframe
suffix = "_realigned"
df = df.join(transformed_points[list('xyz')].rename(columns=lambda x: f"{x}_ccf{suffix}"))

In [10]:
import seaborn as sns

obs = df.query("slice_int==80")
sns.scatterplot(obs, x='z_ccf_realigned', y='y_ccf_realigned', hue='class', legend=None, s=2)
plt.show()

In [ ]:
import seaborn as sns

obs = df.query("slice_int==80")
sns.scatterplot(obs, x='x_ccf_realigned', y='y_ccf_realigned', hue='class', legend=None, s=2)
plt.show()

In [11]:
# add parcellation index
imdata = abc.get_ccf_labels_image(resampled=False)
new_coords = [f"{x}_ccf{suffix}" for x in 'xyz']
df['parcellation_index'+suffix] = imdata[ccf.image_index_from_coords(df[new_coords])]

# add parcellation metadata
ccf_df = pd.read_csv(abc.ABC_ROOT/f"metadata/Allen-CCF-2020/20230630/parcellation_to_parcellation_term_membership.csv")
ccf_df = ccf_df.pivot(index='parcellation_index', columns='parcellation_term_set_name', values='parcellation_term_acronym').astype('category')
df = df.join(ccf_df[['division','structure','substructure']].rename(columns=lambda x: f"parcellation_{x}{suffix}"),
             on='parcellation_index'+suffix)

In [12]:
df.to_parquet("/data/abc_realigned_metadata_thalamus-boundingbox.parquet")

In [12]:
obs = df.query("slice_int==80")
sns.scatterplot(obs, x='z_ccf_realigned', y='y_ccf_realigned', hue='parcellation_substructure_realigned', legend=None, s=2)
plt.show()

### images

In [18]:
import ccf_transforms as ccft
from importlib import reload
reload(ccft)

In [13]:
# imdata = abc.get_ccf_labels_image()
img_transform = sd.transformations.Scale(10e-3*np.ones(3), 'xyz')
labels = sd.models.Labels3DModel.parse(imdata, dims='xyz', transformations={'ccf': img_transform})
# regions = [f"{section}_ccf" for section in cells_by_section.keys()]
# ccf_annotation = load_ccf_metadata_table(regions)
sdata.add_labels('ccf_regions', labels)

In [19]:
ngrid = 1100
nz = 76

z_res = 2
img_stack = np.zeros((ngrid, ngrid, nz))
for section in sdata.points.keys():
    i = int(np.rint(int(section)/z_res))
    target = sdata[section]
    source = sdata['ccf_regions']
    scale = 10e-3
    target_img, target_grid_transform = ccft.map_image_to_slice(sdata, imdata, source, target, scale=scale, ngrid=ngrid, centered=False)
    img_stack[:,:,i] = target_img.T

In [21]:
plt.imshow(img_stack[:,:,int(np.rint(int(80)/z_res))].T)

In [23]:
import nibabel
nifti_img = nibabel.Nifti1Image(img_stack, affine=np.eye(4))

# Save the NIfTI1Image object as .nii file
nibabel.save(nifti_img, '/data/labels.nii.gz')

## load everything in spatialdata

In [24]:
df_full = abc.get_combined_metadata()

In [25]:
# df = abc.label_thalamus_spatial_subset(df_full, filter=True)
coords = ['x_section', 'y_section', 'z_section']
ref_subset = abc.get_thalamus_reference_ids()
df = ccf.subset_to_ref_bounds(df_full, coords, ref_subset)

slice_label = 'slice_int'
df[slice_label] = df['z_section'].apply(lambda x: int(x*10))

# helps plotting etc
df = df.assign(**df.select_dtypes(include=["category"]).apply(lambda x: x.cat.remove_unused_categories()).to_dict(orient='series'))

In [26]:

# df_ref = df_full.loc[ref_subset.intersection(df_full.index)]
xy = df[coords[:2]]
df_min = xy.min()
df_max = xy.max()
norm_transform = sd.transformations.Sequence([
    sd.transformations.Translation(np.array([-1*df_min[0], -1*df_max[1]]), 'xy'),
    sd.transformations.Scale(1/(df_max-df_min) * np.array([1, -1]), 'xy')
    # above is compensating for inverted y in quicknii template, should be:
    # sd.transformations.Translation(-1*df_min, 'xy'),
    # sd.transformations.Scale(1/(df_max-df_min), 'xy')
    ])

In [27]:
# create separate coordinate systems for each section
transform_dict = ccf.read_quicknii_file("resources/adjusted_10-10_final.json", scale=25)
cells_by_section = dict()
for name, df_section in df.groupby(slice_label, observed=True):
    ccf_transform = sd.transformations.Affine(transform_dict[name].T, 'xyz', 'xyz')
    # cells_by_section[name] = sd.models.ShapesModel.parse(
    #     df_section[coords].values, geometry=0, radius=10e-6, 
    cells_by_section[str(name)] = sd.models.PointsModel.parse(
#! converting to dask here is essential to preserve dtypes
        dd.from_pandas(df_section, npartitions=1), coordinates=dict(zip('xyz', coords)),
        transformations={'ccf': sd.transformations.Sequence([norm_transform, ccf_transform]),
                         str(name): sd.transformations.Identity()})

In [28]:
#! what is going on here!>? (categoricals lost, maybe others)
# test = sd.models.PointsModel.parse(df.head(), coordinates=dict(zip('xyz', coords)))
# test.head()


In [29]:

img_transform = sd.transformations.Scale(10e-3*np.ones(3), 'xyz')
# transformation entry takes form of [primary coordinate system name]: [transform from intrinsic coords/pixels to that system]

imdata = abc.get_ccf_labels_image()
labels = sd.models.Labels3DModel.parse(imdata, dims='xyz', transformations={'ccf': img_transform})
# del imdata, img

In [30]:
# load ccf region metadata
# (not included/updated in 20230830 version)
ccf_df = (
        pd.read_csv(
        abc.ABC_ROOT/f"metadata/Allen-CCF-2020/20230630/parcellation_to_parcellation_term_membership.csv",
        dtype={'parcellation_term_acronym': 'category'})
          .query("parcellation_term_set_name=='substructure'")
)

instance_key='parcellation_index'
region_key='annotated_element'
# regions = list(sdata.labels.keys())
regions = [f"{section}_ccf" for section in cells_by_section.keys()]
# need to repeat table for every annotated element
obs = pd.concat([ccf_df.assign(**{region_key: x}) for x in regions])
ccf_annotation = sd.models.TableModel.parse(ad.AnnData(obs=obs), region=regions, region_key=region_key, instance_key=instance_key)
#! won't save?
# sdata.table = ccf_annotation

In [31]:

sdata = sd.SpatialData.from_elements_dict(dict(ccf_regions=labels, table=ccf_annotation, **cells_by_section))


### create and add transformed image slices

In [33]:

labels_dict = dict(ccf_regions=labels)
for section in sdata.points.keys():
    target = sdata[section]
    source = sdata['ccf_regions']
    scale = 10e-3
    target_img, target_grid_transform = map_image_to_slice(sdata, imdata, source, target, scale=scale)
    section_labels = sd.models.Labels2DModel.parse(target_img, dims='yx', 
                             transformations={section: target_grid_transform})
    sdata.add_labels(f"{section}_ccf", section_labels)
    # labels_dict["{section}_ccf"] = section_labels

In [34]:
# overwrite=True is broken?
# sdata.write("/scratch/abc_atlas_realigned.zarr")

In [35]:
from colorcet import glasbey, gray
section = '70'
fig, ax = plt.subplots()


ccf_label = 'parcellation_term_acronym'
i_set = np.unique(sdata[f"{section}_ccf"])
ccf_groups = ccf_df.loc[ccf_df[instance_key].isin(i_set), ccf_label].unique()

groups = sdata[section]['subclass'].value_counts().loc[lambda x: x>0].index.compute()
groups = [x for x in groups if not 'NN' in x]
(
    sdata
    .pl.render_points(size=0.02, color='subclass', groups=groups, palette=glasbey)
    .pl.render_labels(palette=gray*4, color=ccf_label, groups=ccf_groups, outline=True)
    .pl.show(coordinate_systems=[section], ax=ax, na_in_legend=False)
)
ax.set_ylim(7.5, 3)
ax.set_xlim(2.5, 8.5)
ax.get_legend().remove()
plt.show()

## testing

### image transform single section

In [ ]:
sdata = sd.SpatialData.from_elements_dict(dict(ccf_regions=labels, table=ccf_annotation, **cells_by_section))

In [ ]:
# test single section

section = '70'
target = sdata[section]
source = sdata['ccf_regions']
scale = 10e-3
target_img, target_grid_transform = map_image_to_slice(source, target, scale=scale)
section_labels = sd.models.Labels2DModel.parse(
    target_img, dims='yx', 
    transformations={section: target_grid_transform})
sdata.add_labels(f"{section}_ccf", section_labels)

#! trimmed dimensions are erased when sdata creates SpatialImage
# section_labels = section_labels[(section_labels != 0).any(dim='x'),
#                (section_labels != 0).any(dim='y')]

In [ ]:
from colorcet import glasbey, gray
section = '70'
fig, ax = plt.subplots()


ccf_label = 'parcellation_term_acronym'
i_set = np.unique(sdata[f"{section}_ccf"])
ccf_groups = ccf_df.loc[ccf_df[instance_key].isin(i_set), ccf_label].unique()

groups = sdata[section]['subclass'].value_counts().loc[lambda x: x>0].index.compute()
groups = [x for x in groups if not 'NN' in x]
(
    sdata
    .pl.render_points(size=0.02, color='subclass', groups=groups, palette=glasbey)
    .pl.render_labels(palette=gray*4, color=ccf_label, groups=ccf_groups, outline=True)
    .pl.show(coordinate_systems=[section], ax=ax, na_in_legend=False)
)
ax.set_ylim(7.5, 3)
ax.set_xlim(2.5, 8.5)
ax.get_legend().remove()
plt.show()

### details of image transform

In [ ]:
import xarray as xr
coords = np.arange(-ngrid/2, ngrid/2)
# use xarray broadcasting to create a centered coord grid
# (dims are named 'xx' etc as matching variable names not allowed)
# (yx order for image layout)
axes = [xr.DataArray(data=scale*coords + target_center[n], coords={i+i: coords}, dims=i+i, name=i) 
        for n, i in enumerate('xy')]
grid = xr.merge(axes[::-1])
grid['z'] = target_z

In [ ]:
target_points = grid.to_array().to_numpy().reshape(3, -1)
source_points = ccf.apply_affine_left(mat, target_points)

target_img = scipy.ndimage.map_coordinates(imdata, source_points[::-1, :] - 0.5, prefilter=False, order=0)

In [ ]:
img = target_img
plt.imshow(img, origin='lower')
plt.show()

### alternative approaches

In [ ]:

# option using sdata PointsModel
# if added to sdata could specify coord system and infer transform, keeping original coords too
# (maybe nicer for creating SpatialImage coords)
source_points = sd.transform(sd.models.PointsModel.parse(target_points.T), transform)

In [ ]:
# no way to selectively index to fill past edges?! 
# (ie where() with dask delayed evaluation?)
# round_to_centers = lambda x: (x - 0.5).round().astype(int)
# ind = round_to_centers(source_points).compute()
# ind[ind<0] = np.nan
# for i in 'xyz':
#     ind[i] = ind[i].where(ind[i] < len(source[i]))
    
# can't index with nan
# x, y, z = (xr.DataArray(ind[i], dims=['i']) for i in 'xyz')
# target_img = source[z, y, x]

In [ ]:
# rounding, reindexing, and array indexing mimics map_coordinates (slowly)
# rounding is necessary so reindexing (on unique) is smaller
round_to_centers = lambda x: (x - 0.5).round() + 0.5
ind = round_to_centers(source_points)
# dask option needed to not complain here (set T or F)
import dask.config
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
# reindex will fill nan for outside points
# (could also do by inequality and index without rounding)
    subset = source.reindex({x: ind[x].unique() for x in ind.columns}).compute()
# defining single shared dim prevents broadcasting
# whereas this broadcasts to a cube array
# subset.sel(z=ind['z'], y=ind['y'], x=ind['x'])
x, y, z = (xr.DataArray(ind[i], dims=['i']) for i in 'xyz')
target_img = subset.loc[z, y, x]

In [ ]:
img = target_img.values.reshape(ngrid, ngrid).T
plt.imshow(img, origin='lower')
plt.show()

In [ ]:
from dask.array import from_array
from spatial_image import SpatialImage

img_arr = xr.DataArray(from_array(img), 
                       coords={i: grid.data_vars[i].values for i in 'xyz'}, dims=['y','x'])


img_spa = SpatialImage(img_arr, attrs=dict(transform={section: sd.transformations.Identity()}))
img_spa

In [ ]:
coords_dict = {i: xr.DataArray(target[i], dims=['i']) for i in 'xy'}
cell_regions = img_spa.sel(**coords_dict, method='nearest', tolerance=0.5).values.astype(int)

In [ ]:
# coords = ['x_ccf','y_ccf','z_ccf']
# cells = target.dropna(subset=coords)
# coords_dict = {i[0]: xr.DataArray(cells[i].dropna()/0.01, dims=['i']) for i in coords}
cells = sd.transform(target, transform)
coords_dict = {i: xr.DataArray(cells[i], dims=['i']) for i in 'xyz'}
cell_regions2 = source.sel(**coords_dict, method='nearest', tolerance=0.5).values

In [ ]:
np.mean(cell_regions2 == cell_regions)

In [ ]:
# LabelsModel replaces existing coordinates with pixel indices!!
# sd.models.Labels2DModel.parse(data=img_arr, )

# same happens if added directly as SpatialImage
# name = f"ccf_{section}"
# sdata.add_labels(name, img_spa)
# sdata[name]

In [ ]:
# fails beyond tolerance rather than fill nan
# coords_dict = {i: xr.DataArray(transformed_coords[i], dims=['i']) for i in 'xyz'}
# transformed_img = source.sel(**coords_dict, method='nearest', tolerance=0.5)